<a href="https://colab.research.google.com/github/alihussainia/AI-Makerspace/blob/test/pycaret-nlp/email_spam_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# installing packages

In [ ]:
!pip install --upgrade pycaret

In [ ]:
!python -m spacy download en_core_web_sm
!python -m textblob.download_corpora

# importing pacakges

In [ ]:
import pandas as pd
from pycaret.nlp import *
from pycaret.utils import enable_colab
enable_colab()

Colab mode enabled.


# preparing dataset

In [ ]:
# getting dataset 
dataset = pd.read_csv('messages.csv').reset_index(drop=True)

In [ ]:
dataset.isna().sum()

subject    62
message     0
label       0
dtype: int64

In [ ]:
dataset.fillna(dataset['subject'].mode().values[0],inplace=True)

In [ ]:
# converting message and subject to lowercase
dataset['message'] = dataset['message'].str.lower()
dataset['subject'] = dataset['subject'].str.lower()

In [ ]:
# creating a copy of the dataset
df = dataset.copy()

In [ ]:
# combining subject and message
df['sub_mssg']=df['subject']+df['message']

In [ ]:
# now we don't need subject and message anymore due to sub_mssg
df.drop('subject',axis=1,inplace=True)
df.drop('message',axis=1,inplace=True)

# setting pycaret

In [ ]:
exp_1 = setup(data = df, target = 'sub_mssg', session_id = 123, log_experiment = True, experiment_name = 'email_1')

Description,Value
session_id,123
Documents,2893
Vocab Size,24656
Custom Stopwords,False


# creating models

- List of available models:
  * 'lda' - Latent Dirichlet Allocation         
  * 'lsi' - Latent Semantic Indexing           
  * 'hdp' - Hierarchical Dirichlet Process
  * 'rp' - Random Projections
  * 'nmf' - Non-Negative Matrix Factorization

In [ ]:
lda = create_model('lda', num_topics = 6, multi_core = True)

In [ ]:
plot_model(lda, plot = 'topic_distribution')

In [ ]:
# custom function for most repetitive topic words
def topics():
  topic_list=lda.show_topics() # returns topics as list
  lst=[]

  for i in range(len(topic_list)):
    lst.extend(topic_list[i][1].split(sep='" + '))

  for i in range(len(lst)):  
    lst[i] = lst[i][7:]

  return list(set([x for x in lst if lst.count(x) > 1]))

In [ ]:
# setting topics as custom stopwords
stop=topics()

In [ ]:
exp_2 = setup(data = df, target = 'sub_mssg', session_id = 123, log_experiment = True, experiment_name = 'email_2_cs', custom_stopwords=stop)

Description,Value
session_id,123
Documents,2893
Vocab Size,24539
Custom Stopwords,True


In [ ]:
lda2 = create_model('lda', num_topics = 6, multi_core = True)

In [ ]:
plot_model(lda2, plot = 'topic_distribution')

In [ ]:
plot_model(lda2, plot = 'frequency', topic_num = 'Topic 1')

In [ ]:
tuned_classification = tune_model(model = 'lda', multi_core = True, supervised_target = 'label')

IntProgress(value=0, description='Processing: ', max=25)

Output()

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,05:55:36
Status,. . . . . . . . . . . . . . . . . .,Finalizing
Step,. . . . . . . . . . . . . . . . . .,Evaluating Classifier With 400 Topics


Best Model: Latent Dirichlet Allocation | # Topics: 4 | Accuracy : 0.9761


In [ ]:
save_model(tuned_classification,'tuned_classification_Model_V1')